# SPLIT !   
PRODUCT CODE

#### 🌟 RESULT  

x column 들의 결측 정도가 line으로 나누었을 때 보다, Product_code로 나누었을 때 더 명확하게 나왔음.  
  
따라서 Product_code로 확인 후 row에 대해서 결측치가 40% 이상 발견되는 column 은 삭제 후 학습

## 1️⃣ Load Data

In [84]:
import pandas as pd
import random
import os
import numpy as np
import warnings
warnings.filterwarnings('ignore')

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)


train_df = pd.read_csv('./train.csv')
test_df = pd.read_csv('./test.csv')

train_x = train_df.drop(columns=['PRODUCT_ID', 'Y_Quality', 'TIMESTAMP'])
train_y = train_df['Y_Class']
test_x = test_df.drop(columns=['PRODUCT_ID', 'TIMESTAMP'])


seed_everything(42) # Seed 고정

## 2️⃣ Split 

In [85]:
def split_PC(train_df, test_df, train_x, test_x):

    pc_v = sorted(train_df['PRODUCT_CODE'].value_counts().index)

    # train
    for idx, i in enumerate(pc_v):
        globals()['pc{}'.format(idx+1)]=train_x[train_x['PRODUCT_CODE']==i]

    # train y
    for idx, i in enumerate(pc_v):
        globals()['pc{}_y'.format(idx+1)]=train_x['Y_Class'][train_x['PRODUCT_CODE']==i]

    # test
    for idx, i in enumerate(pc_v):
        globals()['pc{}_test'.format(idx+1)]=test_x[test_x['PRODUCT_CODE']==i]

    print('done')
    
split_PC(train_df, test_df, train_x, test_x)

done


## 3️⃣ Preproccesing

In [86]:
# pc1
null_count_1=50
pc_null_df = pd.DataFrame(data=pc1.isnull().sum().keys(), columns=['col'])
pc_null_df['null'] = pc1.isnull().sum().values
pc1_null_count_drop_col = pc_null_df['col'][pc_null_df['null']>= null_count_1].tolist()
print(len(pc1_null_count_drop_col))

# pc2
null_count_2=1
pc_null_df = pd.DataFrame(data=pc2.isnull().sum().keys(), columns=['col'])
pc_null_df['null'] = pc2.isnull().sum().values
pc2_null_count_drop_col = pc_null_df['col'][pc_null_df['null']>= null_count_2].tolist()
print(len(pc2_null_count_drop_col))

# pc3
null_count_3=2
pc_null_df = pd.DataFrame(data=pc3.isnull().sum().keys(), columns=['col'])
pc_null_df['null'] = pc3.isnull().sum().values
pc3_null_count_drop_col = pc_null_df['col'][pc_null_df['null']>= 50].tolist()
print(len(pc3_null_count_drop_col))

2125
2225
2225


In [87]:
all_pc_list = [pc1, pc2, pc3, pc1_test, pc2_test, pc3_test]
train_pc_list = [pc1, pc2, pc3]

def line_fillna(all, train):

    # drop

        # 'Y_Class'
    for l in train:
        l.drop(['Y_Class'], axis=1, inplace=True)
    
        # null_count_drop_col
    for l in [all[0], all[3]]:
        l.drop(pc1_null_count_drop_col, axis=1, inplace=True)
    
    for l in [all[1], all[4]]:
        l.drop(pc2_null_count_drop_col, axis=1, inplace=True)

    for l in [all[2], all[5]]:
        l.drop(pc3_null_count_drop_col, axis=1, inplace=True)
        
    
    # fillna : 0
    for l in all:
        l.fillna(0, inplace=True)

    # MinMaxScaler
    #from sklearn.preprocessing import MinMaxScaler
    
    #for l in all:
    #    mmx = MinMaxScaler()
    #    mmx.fit()
    #    l['LINE'] = mmx.transform(l['LINE'])
        

    #for l in all:
    #    mmx = MinMaxScaler()
    #    mmx.fit(['T_31', 'A_31', 'O_31'])
    #    l['PRODUCT_CODE'] = mmx.transform(l['PRODUCT_CODE'])


    # labelEncorder
    from sklearn.preprocessing import LabelEncoder

    for l in all:
        le = LabelEncoder()
        le.fit(['T100304', 'T100306', 'T050304', 'T010306', 'T010305', 'T050307'])
        l['LINE'] = le.transform(l['LINE'])
        

    for l in all:
        le = LabelEncoder()
        le.fit(['T_31', 'A_31', 'O_31'])
        l['PRODUCT_CODE'] = le.transform(l['PRODUCT_CODE'])        


line_fillna(all_pc_list, train_pc_list)

print('done')

done


## 5️⃣ Build model

### 1)_RandomForestClassifier

In [88]:
train_x_pc_list = [pc1, pc2, pc3]
train_y_pc_list = [pc1_y, pc2_y, pc3_y]
test_pc_list = [pc1_test, pc2_test, pc3_test]


def PC_model_fit_predeict(train_x_pc_list):

    from sklearn.ensemble import RandomForestClassifier

    # 1) define model, 2) fit, 3) predict
    for idx ,i in enumerate(train_x_pc_list):
        
        # define model
        rfc = RandomForestClassifier(random_state=42)
        
        # fit
        rfc.fit(i, train_y_pc_list[idx])

        # predict
        globals()['pred_pc{}'.format(idx+1)] = rfc.predict(test_pc_list[idx])
        globals()['df_pred_pc{}'.format(idx+1)] = pd.DataFrame(data = globals()['pred_pc{}'.format(idx+1)],
                                                            columns=['Y_Class'],
                                                            index=test_pc_list[idx].index)
        

    df_pred = [df_pred_pc1, df_pred_pc2, df_pred_pc3]

    submit_array = pd.concat(df_pred, axis=0).sort_index()

    return submit_array

    
submit_csv = pd.read_csv('./sample_submission.csv')
submit_csv['Y_Class'] = PC_model_fit_predeict(train_x_pc_list)['Y_Class']
submit_csv.to_csv('split_PC_drop_col_submission.csv', index=False)

submit_csv['Y_Class'].value_counts()

1    215
0     67
2     28
Name: Y_Class, dtype: int64

### 1)_RandomForestClassifier

In [92]:
train_x_pc_list = [pc1, pc2, pc3]
train_y_pc_list = [pc1_y, pc2_y, pc3_y]
test_pc_list = [pc1_test, pc2_test, pc3_test]


def PC_model_fit_predeict(train_x_pc_list):

    from xgboost import XGBClassifier

    # 1) define model, 2) fit, 3) predict
    for idx ,i in enumerate(train_x_pc_list):
        
        # define model
        XGB = XGBClassifier(n_estimators=500, max_depth=2, learning_rate=0.01)

        # fit
        XGB.fit(i, train_y_pc_list[idx])

        # predict
        globals()['pred_pc{}'.format(idx+1)] = XGB.predict(test_pc_list[idx])
        globals()['df_pred_pc{}'.format(idx+1)] = pd.DataFrame(data = globals()['pred_pc{}'.format(idx+1)],
                                                            columns=['Y_Class'],
                                                            index=test_pc_list[idx].index)
        

    df_pred = [df_pred_pc1, df_pred_pc2, df_pred_pc3]

    submit_array = pd.concat(df_pred, axis=0).sort_index()

    return submit_array

    
submit_csv = pd.read_csv('./sample_submission.csv')
submit_csv['Y_Class'] = PC_model_fit_predeict(train_x_pc_list)['Y_Class']
submit_csv.to_csv('split_PC_submission.csv', index=False)

submit_csv['Y_Class'].value_counts()

[20:32:52] WARNING: /var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_4f_b8pp6bp/croot/xgboost-split_1675119661934/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[20:32:54] WARNING: /var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_4f_b8pp6bp/croot/xgboost-split_1675119661934/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[20:32:54] WARNING: /var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_4f_b8pp6bp/croot/xgboost-split_1675119661934/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly s

1    248
2     49
0     13
Name: Y_Class, dtype: int64